In [1]:
import skcriteria as skc
from skcriteria.core.methods import SKCTransformerABC
import numpy as np
import abc


In [2]:
dm = skc.mkdm(
    matrix=[[7, 5, 35], [5, 4, 26], [5, 6, 28], [1, 7, 30], [5, 8, 30]],
    objectives=[max, max, min],
    weights=[2, 4, 1],
    alternatives=["PE", "JN", "AA", "MM", "FN"],
    criteria=["ROE", "CAP", "RI"],
)

dm

ROE[▲ 2.0] CAP[▲ 4.0] RI[▼ 1.0]
PE          7          5        35
JN          5          4        26
AA          5          6        28
MM          1          7        30
FN          5          8        30
[5 Alternatives x 3 Criteria]

In [21]:
class SKCFilterABC(SKCTransformerABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])

    def __init__(self, **criteria_filters):
        if not criteria_filters:
            raise ValueError()
        self._criteria_filters = criteria_filters

    @property
    def criteria_filters(self):
        return dict(self._criteria_filters)

    @abc.abstractmethod
    def _make_mask(self, matrix, criteria):
        raise NotImplementedError()

    def _transform_data(self, matrix, criteria, alternatives, **kwargs):
        criteria_not_found = set(self._criteria_filters).difference(criteria)
        if criteria_not_found:
            raise ValueError(f"Missing criteria: {criteria_not_found}")

        mask = self._make_mask(matrix, criteria)

        filtered_matrix = matrix[mask]
        filtered_alternatives = alternatives[mask]

        kwargs.update(
            matrix=filtered_matrix,
            alternatives=filtered_alternatives,
            dtypes=None,
        )
        return kwargs


class SKCFilterOperatorABC(SKCFilterABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])

    @property
    @abc.abstractmethod
    def _filter(self, arr, cond):
        raise NotImplementedError()

    def _make_mask(self, matrix, criteria):
        cnames, climits = [], []
        for cname, climit in self._criteria_filters.items():
            cnames.append(cname)
            climits.append(climit)

        idxs = np.in1d(criteria, cnames)
        matrix = matrix[:, idxs]
        mask = np.all(self._filter(matrix, climits), axis=1)

        return mask


class FilterGT(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.greater


class FilterGE(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.greater_equal


class FilterLT(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.less


class FilterLE(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.less_equal


class FilterEQ(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.equal


class FilterNE(SKCFilterOperatorABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])
    _filter = np.not_equal


class Filter(SKCFilterABC):

    _skcriteria_parameters = frozenset(["criteria_filters"])

    def _make_mask(self, matrix, criteria):
        mask_list = []
        for cname, flt_func in self._criteria_filters.items():
            crit_idx = np.in1d(criteria, cname, assume_unique=False)
            crit_array = matrix[:, crit_idx].flatten()
            crit_mask = np.apply_along_axis(flt_func, axis=0, arr=crit_array)
            mask_list.append(crit_mask)
            
        mask = np.all(np.column_stack(mask_list), axis=1)

        return mask


Filter(ROE=lambda e: e > 1, RI=lambda e: e >= 28).transform(dm)


C0[▲ 2.0] C1[▲ 4.0] C2[▼ 1.0]
PE         7         5        35
AA         5         6        28
FN         5         8        30
[3 Alternatives x 3 Criteria]

In [20]:
dm

ROE[▲ 2.0] CAP[▲ 4.0] RI[▼ 1.0]
PE          7          5        35
JN          5          4        26
AA          5          6        28
MM          1          7        30
FN          5          8        30
[5 Alternatives x 3 Criteria]